In [7]:
(*** hide ***)
 
(*** condition: prepare ***)
#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"
#r "nuget: FSharpAux, 2.0.0"
#r "nuget: FSharpAux.IO, 2.0.0"
#r "nuget: FSharp.Stats, 0.5.0"
#r "../src/BioFSharp/bin/Release/netstandard2.0/BioFSharp.dll"
#r "../src/BioFSharp.IO/bin/Release/netstandard2.0/BioFSharp.IO.dll"
#r "../src/BioFSharp.BioContainers/bin/Release/netstandard2.0/BioFSharp.BioContainers.dll"
#r "../src/BioFSharp.ML/bin/Release/netstandard2.0/BioFSharp.ML.dll"
#r "../src/BioFSharp.Stats/bin/Release/netstandard2.0/BioFSharp.Stats.dll"
open Plotly.NET
open Plotly.NET.Interactive

Error: Plotly.NET.Interactive version 4.2.0 cannot be added because version 4.2.1 was added previously.

## RPKM & TPM
RNA-Seq is a transcriptomics technique, quantifying RNA molecules in a biological sample. When dealing with RNA-sequencing data, normalization is needed to correct technical biases. RPKM and TPM are two metrics that normalize for gene length and sequencing depth. RNA-Sequencing data needs to be normalized for gene length, because longer genes show greater read counts when expressed at the same level.

#### RPKM:

RPKM (Reads per kilobase million) normalization at first determines a scaling factor, by calculating the sum of all reads in a sample and dividing that number by 1,000,000. That scaling factor is used to calculate RPM (Reads per million), by dividing the read counts for each sample with it. To get RPKM, RPM values are divided by genelength in kilobases. RPKM is applied by using the `RNASeq.rpkms` function.


The following dataset will be used as an example for both normalizations.

In [8]:
open BioFSharp.Stats
let rawDataGeneID = seq {"g01";"g02";"g03";"g04";"g05"}
let rawDataGeneLength = seq {150.; 500.; 1500.; 500.; 1500.}
let rawDataGeneCount = seq {16.; 80.; 384.; 50.; 680.}

let rawData = Seq.map3 (fun id gl gc ->  RNASeq.RNASeqInput.Create id gl gc) rawDataGeneID rawDataGeneLength rawDataGeneCount



In [9]:
// rpkm normalization 
open BioFSharp.Stats
let rpkmData = RNASeq.rpkms rawData



#### TPM:

What differentiates TPM (Transcripts per kilobase million) from RPKM is the order of operations. To calculate TPM values, data gets normalized for gene length first. This is achieved by calculating RPK values (reads per kilobase), by dividing the read counts by genelength in kilobases. The sum of all RPK values is divided by 1,000,000, to get a scaling factor. Finally, TPM values are calculated by dividing the RPK values by the scaling factor.
By normalizing for gene length first, the sum of all samples is always 1,000,000, making comparisons of proportions easier.  TPM is applied by using the `RNASeq.tpms` function.

In [10]:
// tpm normalization
open BioFSharp.Stats
let tpmData = RNASeq.tpms rawData



The effects of both normalizations becomes apparent when comparing the relation of the samples 

In [11]:
// visualization of Raw Data, RPKM & TPM
let dataChart = 
    let rawDataCounts =
        rawData
        |> Seq.map (fun input -> input.GeneCount)
    let rawDataKeys =
        rawData
        |> Seq.map (fun input -> input.GeneID) 
    let rpkmDataCounts =
        rpkmData
        |> Seq.map (fun (keys,rpkm) -> rpkm)
    let rpkmDataKeys =
        rpkmData
        |> Seq.map (fun (keys,rpkm) -> keys)
    let tpmDataCounts =
        tpmData
        |> Seq.map (fun (keys,tpm) -> tpm)
    let tpmDataKeys =
        tpmData
        |> Seq.map (fun (keys,tpm) -> keys)
    let chartlist= [ Chart.Column(values = rawDataCounts, Keys = rawDataKeys, Name = "raw data")
                     Chart.Column(values = rpkmDataCounts, Keys = rpkmDataKeys, Name = "RPKM") |> Chart.withXAxisStyle(TitleText = "gene ID")
                     Chart.Column(values = tpmDataCounts, Keys = tpmDataKeys, Name = "TPM")
                     ]
    chartlist
dataChart |> Chart.Grid(nRows = 1, nCols = 3)
|> Chart.withYAxisStyle "Read Counts"

<!-- Plotly chart will be drawn inside this DIV -->

These graphs show how RPKM and TPM correct for technical biases, especially gene length, as the three shorter genes (g01, g02 & g04) are no longer underrepresented. 

Sources: [RNA-Seqblog](https://www.rna-seqblog.com/rpkm-fpkm-and-tpm-clearly-explained/)